# Парсер тикеров Nasdaq

In [196]:
from bs4 import BeautifulSoup
import requests as req
import yfinance as yf
from string import ascii_uppercase

In [223]:
def get_tickers_from_page(url, volume_threshold = 1_500_000):
    resp = req.get(url)
    soup = BeautifulSoup(resp.text, 'html')
    assert  soup.find_all("table")[5]['class'] == ['quotes']
    table = soup.find_all("table")[5]
    assert table.find_all('th')[5].text == 'Volume' # assert that Close value in the 5th column
    table = soup.find_all("table")[5]
    rows = table.find_all('tr')
    tickers = list()
    for row in rows[1:]:
        if float(row.find_all('td')[5].text.replace(',','.').replace('.','')) > volume_threshold: # if Volume is enough high
            tickers.append(row.find_next('a').text)
    return tickers

In [178]:
price_threshold = 100
volume_threshold = 1_000_000

In [224]:
tickers = list()
for c in ascii_uppercase:
    tickers.extend(get_tickers_from_page(f'http://eoddata.com/stocklist/NASDAQ/{c}.htm'))

In [225]:
len(tickers)

268

In [228]:
with open('./data/NASDAQ/largest.txt','w') as f:
    f.write(' '.join(tickers))

In [282]:
x = yf.download(tickers, start="2018-01-02", end="2020-01-01")

[*********************100%***********************]  268 of 268 completed

2 Failed downloads:
- ALGRR: No data found for this date range, symbol may be delisted
- ALGRW: No data found for this date range, symbol may be delisted


In [283]:
x.shape

(504, 1608)

In [288]:
x = x['Close'].drop(['ALGRR','ALGRW','ZM'], axis=1)

In [289]:
x

,AAL,AAPL,AMD,AMZN,ARCC,ATHX,ATRA,ATVI,AVGO,AXAS,...,SQQQ,TOPS,TQQQ,TRNX,VISL,VXRT,WORX,XOG,XSPA,ZN
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,52.990002,172.259995,10.980000,1189.010010,15.920000,1.81,18.100000,64.309998,267.010010,2.56,...,81.639999,46.00,48.613335,495.00,16.000000,5.94,23.750000,14.62,29.000000,2.31
2018-01-03,52.340000,172.229996,11.550000,1204.199951,15.850000,1.82,19.250000,65.309998,269.929993,2.55,...,79.239998,50.00,50.036667,480.00,16.000000,6.05,26.219999,14.80,28.799999,2.35
2018-01-04,52.669998,173.029999,12.120000,1209.589966,15.800000,1.85,18.000000,64.660004,270.019989,2.57,...,78.839996,54.00,50.330002,525.00,16.100000,6.16,27.740000,14.90,27.799999,2.39
2018-01-05,52.650002,175.000000,11.880000,1229.140015,15.790000,1.91,18.500000,66.370003,271.619995,2.46,...,76.480003,50.00,51.849998,495.00,16.200001,6.16,28.500000,14.72,30.400000,2.28
2018-01-08,52.130001,174.350006,12.280000,1246.869995,15.840000,2.03,18.549999,66.629997,272.269989,2.51,...,75.599998,52.00,52.426666,480.00,16.299999,6.27,27.170000,14.28,28.400000,2.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,29.180000,284.269989,46.540001,1789.209961,18.580000,1.26,16.790001,58.810001,320.500000,0.39,...,22.610001,0.85,85.839996,0.90,0.220000,0.30,2.940000,2.08,0.590000,0.19
2019-12-26,29.670000,289.910004,46.630001,1868.770020,18.580000,1.27,16.000000,58.889999,317.980011,0.39,...,22.010000,0.83,88.070000,0.89,0.250000,0.32,2.860000,2.09,0.620000,0.18
2019-12-27,28.440001,289.799988,46.180000,1869.800049,18.629999,1.25,16.510000,59.189999,316.529999,0.38,...,22.090000,0.85,87.809998,0.83,0.250000,0.34,2.860000,2.02,0.610000,0.17


In [295]:
x.isna().any(axis=1)

Date
2018-01-02     True
2018-01-03     True
2018-01-04     True
2018-01-05     True
2018-01-08     True
              ...  
2019-12-24    False
2019-12-26    False
2019-12-27    False
2019-12-30    False
2019-12-31    False
Length: 504, dtype: bool

In [291]:
df1 = x[x.isna().any(axis=1)]

In [296]:
df1.tail()

,AAL,AAPL,AMD,AMZN,ARCC,ATHX,ATRA,ATVI,AVGO,AXAS,...,SQQQ,TOPS,TQQQ,TRNX,VISL,VXRT,WORX,XOG,XSPA,ZN
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-07,30.889999,259.429993,36.279999,1788.199951,18.580000,1.29,13.520,54.549999,311.739990,0.27,...,27.000000,1.23,72.800003,1.54,0.34,0.33,2.89,2.23,1.09,0.21
2019-11-08,30.760000,260.140015,36.290001,1785.880005,18.530001,1.27,13.400,54.299999,313.410004,0.27,...,26.670000,1.06,73.660004,1.76,0.34,0.35,3.07,2.14,1.09,0.21
2019-11-11,30.590000,262.200012,36.310001,1771.650024,18.549999,1.30,13.050,52.520000,312.750000,0.33,...,26.790001,1.01,73.349998,2.09,0.35,0.35,2.92,1.87,1.10,0.21
2019-11-12,29.520000,261.959991,36.709999,1778.000000,18.610001,1.33,13.000,51.950001,315.529999,0.31,...,26.559999,0.95,73.949997,2.13,0.33,0.33,2.95,1.89,1.04,0.21
2019-11-13,28.660000,264.470001,37.520000,1753.109985,18.660000,1.36,13.105,52.330002,313.670013,0.30,...,26.600000,0.95,73.839996,2.03,0.33,0.34,2.89,1.81,1.01,0.21
